# Mapper

In [17]:
#Install Library
!pip install -U giotto-tda

  Using cached scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.8 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.1
    Uninstalling scikit-learn-1.5.1:
      Successfully uninstalled scikit-learn-1.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-survival 0.23.0 requires scikit-learn<1.6,>=1.4.0, but you have scikit-learn 1.3.2 which is incompatible.


In [18]:
# Import all necessary modules
import pandas as pd
import numpy as np
from gtda.time_series import SingleTakensEmbedding
import matplotlib.pyplot as plt
from gtda.plotting import plot_point_cloud, plot_heatmap
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy
from sklearn.cluster import DBSCAN, KMeans, SpectralClustering, Birch, AgglomerativeClustering, MeanShift
from scipy import stats
from gtda.mapper import CubicalCover, make_mapper_pipeline, Eccentricity, Entropy, Projection, plot_static_mapper_graph,plot_interactive_mapper_graph


In [2]:
dlbcl = pd.read_csv('/content/DLBCL.csv')
dlbcl

,time,status,one,a1,a2,a3,a4,a5,a6,a7,...,a7390,a7391,a7392,a7393,a7394,a7395,a7396,a7397,a7398,a7399
0,4.0006,0,1,-0.2210,-0.17860,-0.05025,-0.1922,-0.2944,0.75440,1.0030,...,0.20710,-0.74500,-0.06380,-0.12240,0.72480,-0.13440,0.520300,0.8013,0.939800,0.04403
1,4.9009,0,1,-0.1946,-0.22380,-0.12600,-0.1614,-0.1994,-0.32301,0.2532,...,0.22860,-0.14917,0.31320,-0.15710,0.15220,-0.33700,-0.254000,0.2317,-0.022500,-0.20954
2,5.6003,0,1,-0.3179,-0.15850,-0.48560,-0.3483,-0.4490,-0.17260,0.2333,...,0.63400,-0.36613,-0.26613,-0.39330,-1.18100,0.08445,-0.112900,0.6758,-0.314630,-0.06200
3,12.1010,0,1,0.4269,0.26242,1.70900,0.7984,0.4512,0.31030,0.5402,...,0.31650,0.34965,0.14732,0.29881,-0.00655,0.01305,0.433610,0.2822,0.119900,0.19310
4,0.6001,1,1,0.5116,0.79760,1.46400,1.0520,1.0410,-0.22900,1.4150,...,-0.05970,0.05024,0.66020,-0.03310,-0.20860,-0.11076,0.215510,-0.3900,-0.309100,-0.08185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0.3009,1,1,-0.2227,-0.27700,-0.32350,-0.6641,-0.7011,-0.11510,0.4525,...,-0.31110,0.09790,0.33086,0.38280,0.23150,1.21700,-0.106800,0.2461,0.202600,0.31298
236,3.1003,1,1,-0.4175,-0.31630,-0.48310,-0.4963,-0.5528,-0.27380,-0.0632,...,0.63180,-0.03500,0.82470,0.52020,-0.56680,-0.04245,-0.024525,0.8613,0.182430,-0.04760
237,7.7001,0,1,-0.2064,-0.00940,-0.15120,0.0637,-0.0390,0.20170,0.1626,...,0.59830,0.48100,-0.01767,0.04740,-0.62020,0.13630,-0.132950,0.1046,0.223800,0.24470
238,2.7005,1,1,0.1602,-0.03630,-0.34550,-0.1353,-0.1539,0.13063,-1.4390,...,0.05921,0.05425,0.03089,-0.34140,-2.75600,0.12474,0.148970,0.2447,0.072645,0.24851


In [5]:
dlbcl_sort = dlbcl.sort_values(by=['time'])
del dlbcl_sort['status']
del dlbcl_sort['one']
dlbcl_sort

,time,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a7390,a7391,a7392,a7393,a7394,a7395,a7396,a7397,a7398,a7399
40,0.000540,0.8093,0.8779,0.88020,1.0090,0.9668,0.09564,-0.8470,-1.2370,-1.11300,...,-0.25820,-0.162700,0.10906,-0.2193,-0.23679,0.18460,-0.71050,-1.3540,-0.54260,0.146610
136,0.000641,0.3805,0.1189,0.29570,0.2999,0.2721,0.01290,-0.2178,-0.8472,-0.11602,...,0.97130,0.044500,0.11788,0.1989,-0.83690,1.73600,-0.25120,-0.9063,0.14920,0.771900
202,0.000698,-0.6366,-0.9098,-0.72110,-0.8628,-0.9986,0.12900,0.0368,0.1858,-0.97560,...,-0.68210,-0.130700,-0.15900,-0.4575,1.45700,0.49980,0.72510,1.3330,1.26600,-0.109740
91,0.000914,-0.5863,-0.8140,-0.55730,-0.5488,-1.0540,-0.02690,0.2357,0.3907,-0.14860,...,0.48190,0.578000,0.49550,0.3791,0.46460,0.04275,0.36465,-1.1110,-0.68550,0.320650
23,0.000985,-0.3974,-0.6567,-0.91480,-0.3516,-0.4891,0.12610,0.8612,1.1280,0.72960,...,0.09820,-0.786900,-0.34700,-0.5134,0.62360,0.78990,-0.63770,-0.2238,-0.26860,0.794900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,16.800000,0.5957,0.5635,0.54200,0.5749,0.5725,-0.47300,1.1010,1.2840,1.84900,...,0.22950,0.323000,-0.30290,0.3211,0.05490,-0.08905,0.02185,-2.2380,-1.10200,-0.410500
30,16.900000,0.3164,0.2358,0.33700,0.3887,0.2354,-0.06414,-0.7633,0.0604,0.22920,...,0.08180,-0.204820,0.08795,-1.6140,1.67100,1.11800,-0.80610,1.0070,0.43490,-0.080395
228,17.400000,-0.1024,-0.0962,-0.02325,-0.1812,-0.1550,0.47800,0.3131,-1.1450,-0.53160,...,-0.36150,-1.917000,-0.69010,-0.1353,-0.49650,-0.02184,-0.23880,0.1218,0.10360,0.045950
233,19.801000,0.0997,0.5154,0.53560,0.2661,0.3412,0.82860,0.6456,0.4848,0.92390,...,1.15600,0.912800,0.04744,0.5070,-0.15471,-0.03231,0.11980,-0.2109,0.11357,0.052360


In [21]:
# Define filter function – can be any scikit-learn transformer
filter_func = Projection(columns=['time'])


# Define cover
cover = CubicalCover(n_intervals=14, overlap_frac=0.55)
# Choose clustering algorithm – default is DBSCAN
clusterer = DBSCAN()
#clusterer = KMeans(n_clusters=2)

# Configure parallelism of clustering step
n_jobs = 1

# Initialise pipeline
pipe = make_mapper_pipeline(
    filter_func=filter_func,
    cover=cover,
    clusterer=clusterer,
    verbose=False,
    n_jobs=n_jobs
)

fig = plot_static_mapper_graph(pipe, dlbcl_sort, color_data=dlbcl_sort, color_features=filter_func)
fig.show(config={'scrollZoom': True})

In [22]:
graph = pipe.fit_transform(dlbcl_sort)
node_elements = graph.vs["node_elements"]

In [23]:
low = [0, 1]
list_df = []
for t in low:
  df = dlbcl_sort.iloc[node_elements[t]]
  list_df.append(df)

data_low = pd.concat(list_df)
data_low

,time,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a7390,a7391,a7392,a7393,a7394,a7395,a7396,a7397,a7398,a7399
40,0.000540,0.8093,0.8779,0.8802,1.0090,0.9668,0.09564,-0.8470,-1.23700,-1.11300,...,-0.2582,-0.1627,0.10906,-0.2193,-0.23679,0.18460,-0.71050,-1.3540,-0.5426,0.14661
136,0.000641,0.3805,0.1189,0.2957,0.2999,0.2721,0.01290,-0.2178,-0.84720,-0.11602,...,0.9713,0.0445,0.11788,0.1989,-0.83690,1.73600,-0.25120,-0.9063,0.1492,0.77190
202,0.000698,-0.6366,-0.9098,-0.7211,-0.8628,-0.9986,0.12900,0.0368,0.18580,-0.97560,...,-0.6821,-0.1307,-0.15900,-0.4575,1.45700,0.49980,0.72510,1.3330,1.2660,-0.10974
91,0.000914,-0.5863,-0.8140,-0.5573,-0.5488,-1.0540,-0.02690,0.2357,0.39070,-0.14860,...,0.4819,0.5780,0.49550,0.3791,0.46460,0.04275,0.36465,-1.1110,-0.6855,0.32065
23,0.000985,-0.3974,-0.6567,-0.9148,-0.3516,-0.4891,0.12610,0.8612,1.12800,0.72960,...,0.0982,-0.7869,-0.34700,-0.5134,0.62360,0.78990,-0.63770,-0.2238,-0.2686,0.79490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,4.100800,0.5350,0.6290,0.6694,0.7037,0.6808,-0.40580,1.4620,1.71900,1.42900,...,-0.1859,0.1512,-0.16070,0.3090,0.68500,-0.22680,-0.46150,-0.7147,-0.5769,-1.30200
102,4.200400,-0.1711,-0.4117,-0.3028,-0.2085,-0.2810,-0.14440,0.3162,-0.34896,-0.27290,...,-0.1175,-0.2647,-0.25150,0.1710,0.18790,-0.15220,-0.61670,0.4101,0.2512,0.78700
195,4.300500,-0.2771,-0.5644,-0.2967,-0.1404,-0.5131,0.29400,-0.0205,-1.63000,-1.23600,...,-0.5699,0.9294,0.88110,0.2863,-0.34740,0.37778,0.20754,0.4272,1.2430,0.19980
16,4.300800,-0.3059,-0.3671,-0.2337,-0.3321,-0.5942,-0.27430,-1.2060,-1.63600,-0.65890,...,-0.6190,0.0944,-0.27960,-0.0141,-0.36570,-0.36560,0.81490,-0.0387,0.3227,-0.10221


In [24]:
high = [8, 9, 10, 11, 12, 13]
list_df = []
for t in high:
  df = dlbcl_sort.iloc[node_elements[t]]
  list_df.append(df)

data_high = pd.concat(list_df)
data_high

,time,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a7390,a7391,a7392,a7393,a7394,a7395,a7396,a7397,a7398,a7399
141,11.600,-0.3104,-0.28860,-0.21530,0.0549,0.0643,0.13540,-0.4883,-0.5018,-0.50487,...,0.83430,0.130900,2.27800,-0.74840,-0.21720,-1.97300,-0.40597,-0.52916,-0.64650,-0.496900
208,11.700,-0.1725,-0.03870,-0.04135,-0.0242,-0.1028,0.05620,0.4311,0.5086,-0.05090,...,-0.28810,0.034300,-0.23700,-0.01950,1.14400,0.21990,0.12680,0.89290,0.80240,-0.065335
3,12.101,0.4269,0.26242,1.70900,0.7984,0.4512,0.31030,0.5402,0.5965,0.43840,...,0.31650,0.349650,0.14732,0.29881,-0.00655,0.01305,0.43361,0.28220,0.11990,0.193100
93,12.201,-0.8415,-0.77180,-1.00700,-0.8729,-0.8435,-0.10820,0.2266,0.1404,-0.58625,...,-0.46699,0.283900,-0.29490,0.42980,-0.28024,-0.07995,-0.20867,1.27300,0.24850,-0.444100
151,12.300,0.6233,0.62670,0.59140,0.7127,0.6964,0.02200,2.2990,2.5080,2.76400,...,-0.40700,-0.179200,0.15220,-0.33400,0.43470,0.12323,-0.17140,-2.18600,-0.04497,-1.595000
110,13.300,-1.2970,-1.21000,-1.27500,-1.1570,-1.2470,-0.15864,-0.5734,-0.1393,-0.07460,...,0.17510,-0.121190,-0.15222,-0.69690,0.65090,-0.21404,-0.31419,0.45560,-0.19814,-0.147110
115,13.801,0.5953,0.66740,0.72360,0.5810,0.7213,0.15630,-0.5330,-1.5490,-1.24000,...,-0.18904,-0.456200,0.13700,-0.69790,0.09000,0.24120,-0.26150,-1.17900,-0.95040,-0.354500
57,14.201,1.2220,1.20000,1.26300,0.9091,1.1670,0.11080,0.0606,0.1956,-0.42370,...,0.50470,-0.051095,0.38240,0.08370,-0.24730,-0.09791,0.99200,0.04910,-0.10140,-0.051700
56,14.400,0.0427,-0.11590,-0.31432,0.0841,-0.0127,-0.10620,-0.2868,-0.3406,-0.70270,...,0.54140,0.281400,0.25680,-0.05540,-0.58350,-0.25400,-0.31640,0.09210,-0.36120,0.757300
137,14.600,0.3607,0.26920,0.46450,0.3791,0.3185,0.01900,-0.5660,-1.3780,0.09215,...,-0.57820,0.111000,0.03655,0.75820,0.21070,0.29160,-0.36610,-2.35700,-2.61300,0.506100


In [25]:
data_low.to_excel('data_low.xlsx')
data_high.to_excel('data_high.xlsx')

In [3]:
data_low=pd.read_excel('/content/data_low.xlsx')
data_high=pd.read_excel('/content/data_high.xlsx')

In [6]:
t_stat = []                                       #list kosong untuk menyimpan nilai t-hitung
p_value = []                                      #list kosong untuk menyimpan p-value
sign_ = []                                        #list kosong untuk menyimpan keputusan signifikan atau tidak
sig_feature = []                                  #list kosong untuk menyimpan feature yang sedang diuji-t
data_ttest = pd.DataFrame()
for k in dlbcl_sort.keys()[1:]:
  t, p = stats.ttest_ind(data_low[k], data_high[k])      #uji t
  t_stat.append(t)
  p_value.append(p)
  if p<0.0001:                                            #aturan alpha bisa disesuaikan 0.001 dpt 32
    sig_feature.append(k)
    sign_.append('significant')
  else:
    sign_.append('not significant')


data_ttest['feature'] = dlbcl_sort.keys()[1:]
data_ttest['t-stat'] = t_stat
data_ttest['p-value'] = p_value
data_ttest['decision'] = sign_
data_ttest

,feature,t-stat,p-value,decision
0,a1,-2.004918,0.046214,not significant
1,a2,-2.243358,0.025884,not significant
2,a3,-2.196913,0.029085,not significant
3,a4,-1.729927,0.085065,not significant
4,a5,-2.000836,0.046656,not significant
...,...,...,...,...
7394,a7395,0.818415,0.414018,not significant
7395,a7396,1.864947,0.063539,not significant
7396,a7397,1.864591,0.063589,not significant
7397,a7398,2.223212,0.027233,not significant


In [28]:
len(sig_feature)

52

In [29]:
sig_feature

['a390',
 'a391',
 'a1055',
 'a1062',
 'a1259',
 'a1456',
 'a1776',
 'a1825',
 'a1829',
 'a1834',
 'a1839',
 'a1841',
 'a1901',
 'a1986',
 'a2458',
 'a2459',
 'a2532',
 'a2672',
 'a2993',
 'a3111',
 'a3313',
 'a3350',
 'a3357',
 'a3694',
 'a3796',
 'a4131',
 'a4143',
 'a4205',
 'a4248',
 'a4569',
 'a4570',
 'a4571',
 'a4594',
 'a4816',
 'a5607',
 'a5608',
 'a5628',
 'a5647',
 'a5762',
 'a5820',
 'a5948',
 'a5950',
 'a5952',
 'a5953',
 'a6453',
 'a6528',
 'a6581',
 'a6628',
 'a6733',
 'a6757',
 'a6896',
 'a7339']

# Survival Analysis

In [30]:
!pip install lifelines

In [31]:
!pip install scikit-survival==0.22.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 46.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-survival
    Found existing installation: scikit-survival 0.23.0
    Uninstalling scikit-survival-0.23.0:
      Successfully uninstalled scikit-survival-0.23.0


In [15]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from lifelines import LogLogisticAFTFitter, LogNormalAFTFitter, WeibullAFTFitter
from sklearn.model_selection import train_test_split
from sksurv.ensemble import RandomSurvivalForest
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.inspection import permutation_importance
from scipy import stats
from lifelines.utils import k_fold_cross_validation
%pylab inline
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
surv_important = dlbcl[sig_feature]
surv_important['time'] = dlbcl['time']
surv_important['status']=dlbcl['status']

<ipython-input-7-6a2d615855ca>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  surv_important['time'] = dlbcl['time']
<ipython-input-7-6a2d615855ca>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  surv_important['status']=dlbcl['status']


In [8]:
X_train, X_test, E_train, E_test, Y_train, Y_test = train_test_split(dlbcl[sig_feature], surv_important['status'], surv_important['time'], test_size=0.25, random_state=17)
aft_weibull_imp = WeibullAFTFitter()
aft_weibull_imp.fit(pd.concat([X_train, E_train, Y_train], axis=1), duration_col='time', event_col='status', robust=True)
aft_weibull_imp.print_summary()
print("C-Index for testing= ", aft_weibull_imp.score(pd.concat([X_test, E_test, Y_test], axis=1), scoring_method="concordance_index"))

<lifelines.WeibullAFTFitter: fitted with 180 total observations, 77 right-censored observations>
             duration col = 'time'
                event col = 'status'
          robust variance = True
   number of observations = 180
number of events observed = 103
           log-likelihood = -231.48
         time fit was run = 2024-08-10 08:37:21 UTC

---
                    coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param   covariate                                                                                                         
lambda_ a1055      -0.20       0.82       0.37            -0.93             0.54                 0.39                 1.71
        a1062       0.02       1.02       0.34            -0.65             0.70                 0.52                 2.00
        a1259       0.08       1.09       0.48            -0.85             1.02                 0.43                 2.77
        a1456      -1.40       0.25       0.74            -2.86             0.05                 0.06                 1.05
        a1776       0.14       1.15       0.56            -0.95             1.24                 0.39                 3.45
        a1825      -1.93       0.15       0.91            -3.71            -0.14                 0.02                 0.87
        a1829       0.82       2.28       0.61            -0.37             2.02                 0.69                 7.52
        a1834      -0.21       0.81       0.49            -1.17             0.74                 0.31                 2.10
        a1839       0.25       1.29       0.59            -0.91             1.42                 0.40                 4.13
        a1841       0.40       1.49       0.75            -1.07             1.87                 0.34                 6.47
        a1901      -0.24       0.79       0.40            -1.01             0.54                 0.36                 1.72
        a1986      -0.11       0.90       0.78            -1.63             1.42                 0.20                 4.14
        a2458       0.28       1.32       1.00            -1.67             2.23                 0.19                 9.29
        a2459      -0.09       0.92       0.90            -1.86             1.68                 0.16                 5.39
        a2532      -0.87       0.42       0.62            -2.08             0.34                 0.13                 1.40
        a2672       0.16       1.18       0.87            -1.54             1.86                 0.21                 6.45
        a2993       0.35       1.42       0.54            -0.71             1.40                 0.49                 4.07
        a3111       0.31       1.37       0.35            -0.38             1.00                 0.68                 2.73
        a3313      -0.00       1.00       0.28            -0.56             0.55                 0.57                 1.74
        a3350      -0.03       0.97       0.49            -0.99             0.93                 0.37                 2.54
        a3357       0.06       1.06       0.38            -0.69             0.82                 0.50                 2.26
        a3694       0.31       1.36       0.32            -0.31             0.93                 0.73                 2.54
        a3796       0.08       1.09       0.44            -0.78             0.95                 0.46                 2.58
        a390        0.14       1.15       0.77            -1.37             1.66                 0.25                 5.25
        a391        0.27       1.31       0.24            -0.20             0.73                 0.82                 2.08
        a4131       0.33       1.39       0.13             0.08             0.59                 1.08                 1.80
        a4143       0.35       1.42       0.38            -0.38             1.09                 0.68                 2.98
        a4205      -0.32       0.72       0.39            -1.09           

C-Index for testing=  0.6432197244379986


In [9]:
aft_loglogistic_imp = LogLogisticAFTFitter()
aft_loglogistic_imp.fit(pd.concat([X_train, E_train, Y_train], axis=1), duration_col='time', event_col='status', robust=True)
aft_loglogistic_imp.print_summary()
print("C-Index for testing= ", aft_loglogistic_imp.score(pd.concat([X_test, E_test, Y_test], axis=1), scoring_method="concordance_index"))

<lifelines.LogLogisticAFTFitter: fitted with 180 total observations, 77 right-censored observations>
             duration col = 'time'
                event col = 'status'
          robust variance = True
   number of observations = 180
number of events observed = 103
           log-likelihood = -229.99
         time fit was run = 2024-08-10 08:37:24 UTC

---
                   coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param  covariate                                                                                                         
alpha_ a1055      -0.39       0.67       0.44            -1.26             0.48                 0.28                 1.61
       a1062       0.10       1.10       0.35            -0.59             0.79                 0.55                 2.21
       a1259       0.09       1.09       0.41            -0.73             0.90                 0.48                 2.45
       a1456      -1.54       0.21       0.70            -2.92            -0.16                 0.05                 0.85
       a1776       0.12       1.13       0.49            -0.85             1.09                 0.43                 2.97
       a1825      -1.83       0.16       0.73            -3.25            -0.41                 0.04                 0.67
       a1829       1.02       2.76       0.63            -0.23             2.26                 0.80                 9.58
       a1834      -0.31       0.73       0.46            -1.21             0.59                 0.30                 1.80
       a1839      -0.15       0.86       0.71            -1.54             1.25                 0.21                 3.50
       a1841       0.60       1.83       0.70            -0.76             1.97                 0.47                 7.15
       a1901      -0.21       0.81       0.48            -1.15             0.73                 0.32                 2.08
       a1986      -0.35       0.71       0.86            -2.04             1.34                 0.13                 3.82
       a2458       0.20       1.22       1.07            -1.90             2.29                 0.15                 9.92
       a2459      -0.16       0.85       0.98            -2.09             1.77                 0.12                 5.85
       a2532      -0.69       0.50       0.60            -1.87             0.49                 0.15                 1.63
       a2672       0.64       1.89       0.85            -1.04             2.31                 0.35                10.08
       a2993       0.28       1.32       0.48            -0.66             1.22                 0.52                 3.39
       a3111       0.57       1.77       0.31            -0.04             1.18                 0.96                 3.27
       a3313      -0.11       0.90       0.28            -0.65             0.44                 0.52                 1.55
       a3350      -0.11       0.90       0.63            -1.34             1.12                 0.26                 3.07
       a3357       0.18       1.20       0.35            -0.50             0.86                 0.60                 2.37
       a3694       0.23       1.26       0.28            -0.32             0.78                 0.72                 2.19
       a3796      -0.16       0.85       0.42            -0.99             0.67                 0.37                 1.95
       a390       -0.16       0.85       0.90            -1.91             1.60                 0.15                 4.94
       a391        0.23       1.26       0.28            -0.31             0.78                 0.73                 2.18
       a4131       0.34       1.41       0.13             0.08             0.61                 1.08                 1.83
       a4143       0.56       1.75       0.38            -0.18             1.30                 0.83                 3.67
       a4205      -0.46       0.63       0.41            -1.26             0.35                 0.2

C-Index for testing=  0.6555474981870921


In [10]:
aft_lognormal_imp = LogNormalAFTFitter()
aft_lognormal_imp.fit(pd.concat([X_train, E_train, Y_train], axis=1), duration_col='time', event_col='status', robust=True)
aft_lognormal_imp.print_summary()
print("C-Index for testing= ", aft_lognormal_imp.score(pd.concat([X_test, E_test, Y_test], axis=1), scoring_method="concordance_index"))

<lifelines.LogNormalAFTFitter: fitted with 180 total observations, 77 right-censored observations>
             duration col = 'time'
                event col = 'status'
          robust variance = True
   number of observations = 180
number of events observed = 103
           log-likelihood = -231.31
         time fit was run = 2024-08-10 08:37:26 UTC

---
                   coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param  covariate                                                                                                         
mu_    a1055      -0.33       0.72       0.40            -1.12             0.46                 0.33                 1.59
       a1062       0.12       1.13       0.34            -0.54             0.78                 0.58                 2.19
       a1259       0.27       1.31       0.38            -0.47             1.01                 0.62                 2.74
       a1456      -1.52       0.22       0.73            -2.95            -0.10                 0.05                 0.91
       a1776       0.40       1.49       0.45            -0.48             1.29                 0.62                 3.62
       a1825      -1.93       0.15       0.67            -3.24            -0.61                 0.04                 0.54
       a1829       1.01       2.73       0.51             0.01             2.00                 1.01                 7.39
       a1834      -0.52       0.59       0.42            -1.35             0.31                 0.26                 1.36
       a1839       0.07       1.07       0.65            -1.21             1.34                 0.30                 3.83
       a1841       0.46       1.59       0.57            -0.66             1.58                 0.52                 4.88
       a1901      -0.30       0.74       0.43            -1.14             0.54                 0.32                 1.71
       a1986      -0.55       0.57       0.73            -1.98             0.87                 0.14                 2.40
       a2458       0.15       1.16       0.99            -1.79             2.10                 0.17                 8.15
       a2459      -0.30       0.74       0.87            -2.01             1.40                 0.13                 4.06
       a2532      -0.89       0.41       0.57            -2.01             0.23                 0.13                 1.26
       a2672       0.60       1.82       0.72            -0.81             2.01                 0.44                 7.48
       a2993       0.18       1.20       0.45            -0.69             1.06                 0.50                 2.89
       a3111       0.66       1.93       0.30             0.07             1.24                 1.08                 3.47
       a3313      -0.09       0.91       0.24            -0.56             0.37                 0.57                 1.45
       a3350       0.07       1.07       0.52            -0.95             1.09                 0.39                 2.96
       a3357       0.24       1.27       0.34            -0.43             0.91                 0.65                 2.48
       a3694       0.13       1.14       0.27            -0.41             0.67                 0.67                 1.95
       a3796      -0.13       0.88       0.39            -0.89             0.63                 0.41                 1.88
       a390       -0.40       0.67       0.76            -1.90             1.09                 0.15                 2.98
       a391        0.30       1.35       0.27            -0.23             0.83                 0.80                 2.29
       a4131       0.31       1.37       0.13             0.06             0.56                 1.07                 1.75
       a4143       0.67       1.95       0.33             0.03             1.30                 1.03                 3.68
       a4205      -0.42       0.66       0.36            -1.12             0.29                 0.33 

C-Index for testing=  0.668600435097897


In [11]:
#Buat label RSF
dt = [('cens', '?'), ('time', '<f8')]
Y_ = []
for i, a in enumerate(surv_important['time']):
  if surv_important['status'][i]==1:
    Y_.append((True, a))
  else:
    Y_.append((False, a))

Y_ar = np. array(Y_, dtype=dt)
Y_ar

array([(False, 4.0006e+00), (False, 4.9009e+00), (False, 5.6003e+00),
       (False, 1.2101e+01), ( True, 6.0010e-01), ( True, 3.0007e-01),
       ( True, 4.0023e-01), ( True, 1.2009e+00), ( True, 2.4001e+00),
       ( True, 1.2003e+00), (False, 8.8010e+00), (False, 8.1002e+00),
       (False, 5.2005e+00), (False, 8.1003e+00), ( True, 2.8007e+00),
       ( True, 4.5010e+00), (False, 4.3008e+00), ( True, 2.5007e+00),
       ( True, 1.3001e+00), ( True, 1.0010e-01), ( True, 1.7000e+00),
       (False, 7.2003e+00), ( True, 6.0082e-01), ( True, 9.8548e-04),
       ( True, 3.0002e-01), ( True, 3.0082e-01), ( True, 2.5006e+00),
       (False, 1.0801e+01), (False, 7.7002e+00), (False, 5.9008e+00),
       ( True, 1.6900e+01), (False, 9.7008e+00), (False, 1.0101e+01),
       (False, 1.6002e+00), ( True, 8.0060e-01), ( True, 3.9006e+00),
       ( True, 3.3007e+00), (False, 7.1002e+00), (False, 3.3006e+00),
       (False, 6.8002e+00), ( True, 5.3960e-04), ( True, 1.0062e-01),
       ( True, 2.700

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(surv_important[sig_feature], Y_ar, test_size=0.25, random_state=17)
rsf_dlbcl1 = RandomSurvivalForest(n_estimators=70,
                           min_samples_split=3,
                           min_samples_leaf=10,
                           max_features="sqrt",
                           n_jobs=-1, random_state=1)
rsf_dlbcl1.fit(X_train, Y_train)
c_rsf1 = rsf_dlbcl1.score(X_test, Y_test)
print("C-Index = ", c_rsf1)

C-Index =  0.6809282088469906


In [16]:
result = permutation_importance(rsf_dlbcl1, X_test, Y_test, n_repeats=25, random_state=6)
df_result = pd.DataFrame({k: result[k] for k in (
            "importances_mean",
            "importances_std",)},
    index=X_test.columns,).sort_values(by="importances_mean", ascending=False)

df_result

,importances_mean,importances_std
a4131,0.033474,0.013154
a6733,0.028165,0.008589
a2532,0.023785,0.007335
a2672,0.019260,0.007625
a1825,0.018477,0.008999
a4248,0.015141,0.009961
a5628,0.013662,0.005272
a5950,0.012125,0.007692
a3796,0.010268,0.009694
a1456,0.010094,0.004512
